In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Set root package root directory.
import os
os.environ['ML_FOR_SG_ROOT'] = 'Z:\Dev\Github\ml-for-sg'

# Create shotgun data manager.
from lib.source_generator.shotgun.source_generator import ShotgunSourceGenerator
shotgun_source_generator = ShotgunSourceGenerator()